# Intro
The goal of this notebook is to build an image classifier for images of building's facades (label 'Facade') or interiors (label 'Flat').

Dataset with train images is provided by Happs team (http://namr.com/data/cv/data.zip).

## Module Imports
First of all we need to import all necessary modules:

In [ ]:
import os
import shutil
import numpy as np

from matplotlib import pyplot as plt 

import keras
from keras import backend as K
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.layers import Input, Lambda, Dense, BatchNormalization, Flatten
from keras.layers import Convolution2D, MaxPooling2D, Dropout
from keras.models import Model
from keras.optimizers import Adam, Adagrad, SGD, RMSprop
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

%matplotlib inline

## Initialize common variables
We set random seed and some shared variables

In [ ]:
# Read batch of data
bx, by = next(train_iterator)

# Show images
image_plots(imgs=bx[:20], titles=by[:20], rows=4)

Everything seems to be ok, we have images of interiors and facades as expected. Also we can see that image labels represented in '1-hot-encoded' form, which means that label 'Facade' has code `[1, 0]` and 'Flat' is `[0, 1]`

## Validation Dataset
In order to evaluate performance of the classifier we are building, we should use validation data. It is not appropriate to evaluate performance on the training data.
But in the original data zip file from Happs there are no validation images. 

There are some possible solutions. First of all, we can split train data into 2 subsets with ratio like 80/20 to construct validation dataset. But in our case, training data is already small and we'll need it all to perform training. 

Second option is to mine missing data ourselves, and it is that what was done. Custom validation data was copied from internet and saved to `data/val-custom` directory. It is important to remind, that we will *not use validation data* for training. Validation dataset is used only to evalute perfomance of image classifier.

In [ ]:
# Use custom validation dataset
VAL_PATH = os.path.join(DATA_PATH, 'val-custom')
# Create image iterator
val_iterator = image_gen.flow_from_directory(VAL_PATH, 
                                            batch_size=1, 
                                            target_size=TARGET_SIZE, 
                                            shuffle=False)

In [ ]:
# How many images per class?
class_idx = classes_stat(val_iterator)

# Building Image Classifier
Our task "to classify images into 2 classes" is a classical image recognition task where the state-of-the-art solutions are Convolutional Neural Networks (CNN). 

So we will start with building VGG-like custom CNN.

## VGG-like custom CNN
Our model will have:
- 4 Convolution layers (2 layers with 64 filters, and 2 layers with 32 filters);
- Each block of Convolution layers will have one Max Pool layer;
- On the top we will put 2 Dense non-linear layers; 
- 1 Dense softmax layer to output predictions. 


*Note*: There are no strict theory on how to define hyper-parameters (how many layers-filters-etc do we need), so all parameters below are just some reasonable "start-with" values.


To build the model we will use Keras framework with Tensorflow backend.

In [ ]:
# Helper to build VGG-like CNN model
def build_cnn_model():
    # Prepare input for model with custom input shape
    input_tensor = Input(shape=INPUT_SHAPE)
    
    # Conv block 64 filters
    x = Convolution2D(64, 3)(input_tensor)
    x = Convolution2D(64, 3)(x)
    x = MaxPooling2D()(x)
    
    # Conv block 32 filters
    x = Convolution2D(32, 3)(x)
    x = Convolution2D(32, 3)(x)
    x = MaxPooling2D()(x)
    x = Flatten()(x)
    
    # Dense block for classification
    x = Dense(32, activation='relu')(x)
    x = Dense(32, activation='relu')(x)
    pred_layer = Dense(NUM_CLASSES, activation='softmax', name='predictions')(x)

    # Build and compile model
    model = Model(inputs=input_tensor, outputs=pred_layer)
    model.compile(optimizer=SGD(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
# Create model object
model = build_cnn_model()
model.summary()

In [ ]:
# Setup variables
val_steps = val_iterator.n  # number of val images

# Train the model
model.fit_generator(train_iterator,
                    steps_per_epoch=10, 
                    epochs=3, 
                    validation_data=val_iterator, 
                    validation_steps=val_steps)

# Slow down learning rate and continue to train
model.optimizer.lr = 1E-5
model.fit_generator(train_iterator,
                    steps_per_epoch=10, 
                    epochs=5, 
                    validation_data=val_iterator, 
                    validation_steps=val_steps)

Results are not very satisfying: validation loss and accuracy is stagnating around the same level. 

The fact is that our model has almost 3M parameters and only 50 training images. 

To deal with that, let's use a *"data augmentation"* trick. 

## Data Augmentation
Data augmentation is a technique to add some random transformations (like rotation or zooming) to the original images to generate the new one. Important condition is to keep images realistic enough after transformation. 

Let's change a bit image generator to add some random transformation into the images. That will allow us to "strech" our intial dataset into something bigger!

In [ ]:
# Image generator with random transformations
image_trans_gen = ImageDataGenerator(rescale=1./255, 
                                     channel_shift_range=8,
                                     horizontal_flip=True,
                                     rotation_range=3, 
                                     zoom_range=(1.0, 0.7))

In [ ]:
# Train image iterator with random transformations
train_iterator = image_trans_gen.flow_from_directory(TRAIN_PATH, 
                                               batch_size=BATCH_SIZE, 
                                               target_size=TARGET_SIZE)

In [ ]:
# Let's plot transformed images
bx, by = next(train_iterator)
image_plots(imgs=bx[:20], titles=by[:20], rows=4)

Now we can train the model again:

In [ ]:
model = build_cnn_model()

# Train the model
model.fit_generator(train_iterator,
                    steps_per_epoch=10, 
                    epochs=2, 
                    validation_data=val_iterator, 
                    validation_steps=val_steps)

# Slow down learning rate and continue to train
model.optimizer.lr = 1E-5
model.fit_generator(train_iterator,
                    steps_per_epoch=10, 
                    epochs=5, 
                    validation_data=val_iterator, 
                    validation_steps=val_steps)

Apparently, our results are better than before, but let's see if we can do better!

It could be a chance to try another helpful technique: *transfer learning*.

## Transfer Learning
Transfert learning is a method of building new neural network when we re-use pre-trained weights of other network (more details about transfert learning: https://cs231n.github.io/transfer-learning/).

In out case we will take Convolutional Neural Network VGG16 with weights pre-trained on 'Imagenet' dataset.

In 'Imagenet' there are 1000 classes, but we have 2. So we will adapt the top layers of network to produce predictions for 2 classes, and we will train only these added layers.

## Transfert Learning with VGG16 Network
Let's build adapted VGG16 model:

In [ ]:
def build_vgg_ft_model():
    # Prepare input for model with custom input shape
    input_tensor = Input(shape=INPUT_SHAPE)

    # Build model with pretrained weights and not top layers
    base_model = VGG16(input_shape=INPUT_SHAPE,
                           input_tensor=input_tensor,
                           weights='imagenet',
                           include_top=False)

    # Freeze layers so training will not change its weights
    for layer in base_model.layers: layer.trainable = False

    # Add dense output with num_classes
    x = base_model.output
    x = BatchNormalization()(x)
    x = Flatten()(x)
    x = Dropout(0.5, name='drop')(x)
    x = Dense(32, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dense(32, activation='relu')(x)
    x = BatchNormalization()(x)
    pred_layer = Dense(NUM_CLASSES, activation='softmax', name='predictions')(x)

    # Build model for provided classes
    model = Model(inputs=base_model.input, outputs=pred_layer)
    model.compile(optimizer=Adam(lr=1E-4), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
model = build_vgg_ft_model()
model.summary()

In [ ]:
# Train the model
model.fit_generator(train_iterator, 
                    steps_per_epoch=10, 
                    epochs=5, 
                    validation_data=val_iterator, 
                    validation_steps=val_steps)

This model shows much better results! Indeed, combination of data augmentation and transfer learning allows us to train a good model with very few intial training data.

## Note on Overfitting
Sometimes model's perfomance on validation data drops after several epochs. It was the case with both custom CNN and VGG16-based CNN. 

It is a sign of overfitting, and maybe we could have added 'early stopping' to prevent model from overfitting and save training time, but due to the limit time we will not add this step.

## Save prediction results
In the end let's save trained model's weights so we can reuse it later without need to train it again.

In [ ]:
# Set seed
from numpy.random import seed
seed(42)
from tensorflow import set_random_seed
set_random_seed(42)

# Global variables
BATCH_SIZE = 128
TARGET_SIZE=(224, 224)  # Resize input images to that size
INPUT_SHAPE = TARGET_SIZE + (3,)
NUM_CLASSES = 2  
DATA_PATH = './data/'  # Root data path
TRAIN_PATH = os.path.join(DATA_PATH, 'train')  # Train data path 
VAL_PATH = os.path.join(DATA_PATH, 'val')  # Validation data path

## Helper functions

In [ ]:
def classes_stat(image_iterator):
    """
    Function to print how many items per class has `image_iterator`.
    Returns dictionary `class_idx` to match class index to label
    """
    classes = image_iterator.classes
    class_idx = {v:k for k, v in image_iterator.class_indices.items()}
    for c in np.unique(classes):
        count = np.sum(classes==c)
        print('Class {} ({}): {} items'.format(c, class_idx[c], count))
    return class_idx

In [ ]:
def image_plots(imgs, figsize=(12,8), rows=1, interp=False, titles=None):
    """
    Function to plot images from `imgs` array with optional labels from `titles`.
    Images will be plot in one figure with number of `rows`. 
    """
    f = plt.figure(figsize=figsize)
    cols = len(imgs)//rows if len(imgs) % 2 == 0 else len(imgs)//rows + 1
    for i in range(len(imgs)):
        sp = f.add_subplot(rows, cols, i+1)
        sp.axis('Off')
        if titles is not None:
            sp.set_title(titles[i], fontsize=12)
        plt.imshow(imgs[i], interpolation=None if interp else 'none')

# Dataset Exploration
Let's check how many images do we have in the training dataset. Also let's plot some of the images to have an idea what kind of content is there.

## Train Dataset 

In [ ]:
# Create image generator to read images from directory 
# and rescale pixel values from range [0; 255] to [0;1] 
image_gen = ImageDataGenerator(rescale=1.0/255)
train_iterator = image_gen.flow_from_directory(TRAIN_PATH, 
                                               batch_size=BATCH_SIZE, 
                                               target_size=TARGET_SIZE)

In [ ]:
# How many images per class?
_ = classes_stat(train_iterator)

Indeed we have 2 classes of images. Also it is clear that our dataset is quite small - 50 images total. That could be a challenge to train an image classifier.

To finish with data exploration, we will plot images from one batch of train dataset. 

In [ ]:
# Save weights and model
model.save_weights('./vgg16ft_weights.h5')

In [ ]:
# Load saved weights
model.load_weights('./vgg16ft_weights.h5')

Also let's run prediction of labels on validation dataset and save that as csv file.